このjupyter notebookは，以下の論文3章における日本におけるSci-Hub利用状況分析に用いたプログラムです。

大谷周平, 坂東 慶太. 論文海賊サイト Sci-Hub を巡る動向と日本における利用実態. 情報の科学と技術.2018, 68(10).  https://doi.org/10.18919/jkg.68.10_513

Sci-Hub 2017年ダウンロードデータおよびImpactStory社によるUnpaywallのデータセットを利用して分析しています。

Sci-Hub ダウンロードログデータ
Sci Hub on Twitter: Sci-Hub download log for 2017 year https://twitter.com/Sci_Hub/status/954095639859814402

Unpaywallのデータセット
Priem, Jason and Piwowar, Heather. (2018) The Unpaywall Dataset. DOI: https://doi.org/10.6084/m9.figshare.6020078


必要なライブラリのインポート

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import folium
import sqlite3
import math
import matplotlib
import matplotlib.pyplot as plt
import datetime as dt
import pickle
import matplotlib.font_manager
sns.set(font='IPAexGothic')

2017年6月の時間帯別利用状況


In [3]:
dbname = "data_anlytics.db"
conn = sqlite3.connect(dbname)
conn.row_factory = sqlite3.Row
c = conn.cursor()

In [19]:
df=pd.read_sql_query("select * from 'scihubjapanoadoi'", conn, index_col=["date"], parse_dates=["date"])

In [5]:
df.shape

(1271346, 25)

In [9]:
def preprocess_month(x):#マルチインデックスを作成して、月・時間単位の分析を行う
    df_m = x.set_index([x.index.month, x.index.day,x.index.weekday_name,x.index.hour,x.index,x.index.year])
    df_m.index.names = ['month', 'day','weekday','hour','date','year1']#yearは重複するのでyear1に。
    return df_m

In [20]:
a=df["2017/06/01":"2017/06/28"]#4週間分にデータを限定
a.index = a.index.tz_localize('UTC')
a.index=a.index.tz_convert('Asia/Tokyo')
a.index.tz_localize(None) #UTCから日本時間に変換。タイムゾーンを削除してローカル日時に変換


DatetimeIndex(['2017-06-01 09:01:35', '2017-06-01 09:01:35',
               '2017-06-01 09:01:39', '2017-06-01 09:01:43',
               '2017-06-01 09:03:01', '2017-06-01 09:03:23',
               '2017-06-01 09:05:09', '2017-06-01 09:07:11',
               '2017-06-01 09:07:57', '2017-06-01 09:10:29',
               ...
               '2017-06-29 08:54:22', '2017-06-29 08:54:40',
               '2017-06-29 08:54:57', '2017-06-29 08:55:08',
               '2017-06-29 08:55:14', '2017-06-29 08:55:31',
               '2017-06-29 08:56:24', '2017-06-29 08:57:20',
               '2017-06-29 08:58:34', '2017-06-29 08:58:44'],
              dtype='datetime64[ns]', name='date', length=119578, freq=None)

In [22]:
b=a["doi"].groupby([a.date.dt.weekday.rename('weekday'),a.date.dt.hour.rename('hour')]).count()
c={}
counter="None"
counter_value=None
counter_list=[]
for column_name, item in b.iteritems():
    if counter != column_name[0]:
        c[counter]=counter_list
        counter_list=[]
        counter = column_name[0]
        counter_value=item
        counter_list.append(counter_value)
    else:
        counter_value=item
        counter_list.append(counter_value)
c[counter]=counter_list
del c["None"]
d=pd.DataFrame(c)
d=d.ix[:,["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]]
d=d.rename(columns={"Monday":"月","Tuesday":"火","Wednesday":"水","Thursday":"木","Friday":"金","Saturday":"土","Sunday":"日"})
sns.heatmap(d,cmap="Reds")

AttributeError: 'DataFrame' object has no attribute 'date'